In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
image_path = "../images/before/sudoku-puzzle-863979.jpg"

In [80]:
#-------------------------------------------------------------------------------------------------------
def create_image(height,width,grayScale = False,grayChannel = False, whiteImage = True):
    """
    This will create white or a black image as per requirement.
    Args:
        height      -> Tells about the height of image. Let say height = 512(pixels).
        width       -> Tells about the width of image. Let say width = 512(pixels).
        grayScale   -> Whether we want grayScale image or not.
        grayChannel -> Whether we want channel or not.
        whiteImage  -> Whther we want white image/ black image.
        
        
    Return:
        It will return an numpy.ndarray of shape (height,width,_), which is basically denotes an image.
    """   
    if whiteImage: # Check whether user want whiteImage or not.
        
        if grayScale: #GrayScale or not
            if grayChannel: #grayChannel or not
                return 255. * np.ones((height,width,1),np.uint8) # return an numpy.ndarray
            else:
                return 255. * np.ones((height,width),np.uint8) # return an numpy.ndarray
        else:
            return 255. * np.ones((height,width,3),np.uint8) # return an numpy.ndarray
        
    else:
        if grayScale:
            if grayChannel:
                return np.zeros((height,width,1),np.uint8)
            else:
                return np.zeros((height,width),np.uint8)
        else:
            return np.zeros((height,width,3),np.uint8)

#-------------------------------------------------------------------------------------------------------        
def show_image(image,image_title: str = ""):
    # import opencv
    import cv2
    
    cv2.imshow(image_title,image) # Display image
    cv2.waitKey(0) # Wait for key to be pressed(any key)
    cv2.destroyAllWindows() # close the program
    
#-------------------------------------------------------------------------------------------------------  
def preprocess(image,
               save_image = False,
               dilate_image = False,
               filepath : str = "",
               filename : str = "",
               extension : str = ""
              ):
    
    import cv2
    # Read image using open-cv
    image = cv2.imread(image)
    # Make BGR image to GRAY Scale image
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    if save_image:
        cv2.imwrite(filepath + "/" + filename + extension,image)
        
    # make image blur
    blur_image = cv2.GaussianBlur(gray_image,(9,9),0)
    
    #
    threshold_image = cv2.adaptiveThreshold(blur_image,
                                            255,
                                            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY,
                                            11,
                                            2
                                           )
    # Invert the colors of image because we need to find grid edges.
    output_image = cv2.bitwise_not(threshold_image,threshold_image)
    
    # dilate image
    if dilate_image:
        
        kernel = [[0.,1.,0.],[1.,1.,1.],[0.,1.,0.]]
        kernel = np.array(kernel)
        
        output_image = cv2.dilate(output_image,kernel)
        
    return output_image

#----------------------------------------------------------------------------------------------------
def GRAY_RGB_GRAY(image,GRAY_RGB = True):    
    if GRAY_RGB and (len(image.shape) == 2 or image.shape[2] == 1):
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
        return image
    else:
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        return image
    
#------------------------------------------------------------------------------------------------------
def findContours(image):
    contours,hierarchy = cv2.findContours(image,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    return contours
#-------------------------------------------------------------------------------------------------------
def plotContours(image,showImage = False):
    
    contours = findContours(image)
    image = GRAY_RGB_GRAY(image)
    drawContours = cv2.drawContours(image,contours,-1,(226,3,255),3)
    
    if showImage:
        show_image(drawContours)
        
    return drawContours

#--------------------------------------------------------------------------------------------------------
def cal_corners(image):
    import operator
    
    contours = findContours(image)
    
    contours = sorted(contours,key = cv2.contourArea,reverse=True)
    
    for_largest = contours[0]
    
    bottom_r, _ = max( enumerate([ coordinate[0][0] + coordinate[0][1] for coordinate in for_largest]), key=operator.itemgetter(1) )
    top_l, _ = min( enumerate([ coordinate[0][0] + coordinate[0][1] for coordinate in for_largest]), key=operator.itemgetter(1) )
    bottom_l, _ = min( enumerate([ coordinate[0][0] - coordinate[0][1] for coordinate in for_largest]), key=operator.itemgetter(1) )
    top_r, _ = max( enumerate([ coordinate[0][0] - coordinate[0][1] for coordinate in for_largest]), key=operator.itemgetter(1) )
    
    return [ for_largest[top_l][0],for_largest[top_r][0],for_largest[bottom_r][0],for_largest[bottom_l][0] ]

#-----------------------------------------------------------------------------------------------------------
def display_points(image,radius = 10, showImage=False):
    corners = cal_corners(image)
    image = GRAY_RGB_GRAY(image)
    
    for point in corners:
        center = tuple(int(x) for x in point)
        image = cv2.circle(image,center,radius,(226,0,255), -1)

    if showImage:
        show_image(image)
    return image

#-----------------------------------------------------------------------------------------------------------


In [70]:
image = preprocess(image_path)

In [40]:
show_image(image)

In [38]:
s = GRAY_RGB_GRAY(image)

In [39]:
show_image(s)

In [48]:
a = plotContours(image,True)

In [61]:
corners = cal_corners(image)